In [1]:
import copy
import csv
import pandas as pd

#### 1) conditonsの条件に該当しない項目をNoneとする

In [2]:
def set_none(d, case):
    
    for key in d :
        if 'conditions' in d[key]:            
            set_typical = False
            
            # conditionsの中に全ての項目が条件を満たす組合せがある場合には、typicalを設定に該当
            for condition in d[key]['conditions']:
                if all(case[condition_key] == condition[condition_key] for condition_key in condition): 
                    set_typical = True
                    
            # typicalを設定に該当しない場合には、Noneとする
            if set_typical == False:
                case[key] = None

#### 2) 基準ケース作成

In [3]:
def make_standard_case(d):
    
    standard_case = {}
    
    # 基準ケースの作成
    for key in d:
        standard_case[key] = d[key]['typical']

    set_none(d, standard_case)
                    
    return standard_case

#### 3) リスト要素の追加・削除

In [4]:
def add_and_remove_list(list, setting):
    
    # リスト要素の追加
    if 'extra' in setting:
        for element in setting['extra']:
            if element not in list:
                list.append(element)
    
    # リスト要素の削除
    if 'exception' in setting:
        for element in setting['exception']:
            if element in list:
                list.remove(element)
    
    return sorted(list)

#### 4) 作成済ケースの確認

In [5]:
def check_existence(cases):
    
    existing_cases = {}
    
    for i, case in enumerate(cases):
    
        # 作成済ケースに一致するケースがある場合は、一致するケースの番号を取得
        t_case = tuple(case.values())
        if t_case in existing_cases:
            case['existing_case'] = existing_cases[t_case] + 1

        # 作成済ケースに一致するケースがない場合は、作成済ケースに追加
        else:
            existing_cases[t_case] = i


#### 5) 条件を満たすようconditionに規定された項目の値を設定

In [6]:
def meet_condition(d, case, key):
    
    # リストの1番目の条件を満たすよう、リスト中の項目を設定する
    for condition_key in  d[key]['conditions'][0]:
        case[condition_key] = d[key]['conditions'][0][condition_key]    
        if 'conditions' in d[condition_key]:
            meet_condition(d, case, condition_key)
            
    return case

#### 6) 要素選択の項目の設定値リスト作成

In [7]:
def make_list_for_options(cases, standard_case, key, d):
    
    # ケース設定用のリスト作成
    base_options = d[key]['options']
    temp = [0] + base_options + [max(base_options) + 1]
    
    options = add_and_remove_list(temp, d[key])
    
    return options

#### 7) 値入力の項目の設定値リスト作成

In [8]:
def make_list_for_values(cases, standard_case, key, d):
    
    # ケース設定用のリスト作成
    temp = [
        d[key]['min'] - d[key]['dec'],
        d[key]['min'],
        d[key]['typical'],
        d[key]['max'],
        d[key]['max'] + d[key]['dec']
    ]
    values = add_and_remove_list(temp, d[key])
    
    return values

#### 8) ケース作成

In [9]:
def make_cases(standard_case, d):
    
    cases = []

    # ケースの生成
    for key in d:
        
        #要素選択の項目
        if d[key]['input_method'] == 'choose_from_options' :
            list = make_list_for_options(cases, standard_case, key, d)
        
        #値入力の項目
        elif d[key]['input_method'] == 'input_value' :
            list = make_list_for_values(cases, standard_case, key, d)

        # ケース設定
        for element in list:
            case = copy.deepcopy(standard_case)    
            case[key] = element

            # conditionsを含む場合
            if 'conditions' in d[key]:
                case = meet_condition(d, case, key)

            set_none(d, case)
            
            cases.append(case)

    return cases

#### 9) テストケース出力

In [10]:
def output_cases(keys_and_settings):

    # 基準ケースの作成
    standard_case = make_standard_case(keys_and_settings)

    # テスケースの作成
    cases = []
    cases += [standard_case]
    cases += make_cases(standard_case, keys_and_settings)    

    # 重複確認
    check_existence(cases)   

    # csv出力
    keys = list(keys_and_settings.keys()) + ['existing_case']
    with open("test_lv1_to_lv2_cases.csv", "w", encoding="Shift_jis") as csv_test_lv1_to_lv2_cases:
        csv_test_lv1_to_lv2_cases.write(','.join(keys) + '\n')

        for case in cases:
            row = [(str(case[key]) if key in case else '') for key in keys]
            csv_test_lv1_to_lv2_cases.write(','.join(row) + '\n')

#### 10) 項目設定

・床面積のtypicalは、WEBプログラムのデフォルト値を参考に設定。  
・U値のtypicalは、6地域の仕様基準を参考に設定。  
・窓のη値のtypicalは、金属製建具の二層複層ガラスを参考に設定。  
・窓のf値のtypicalは、「当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法」のデフォルト値を使用する場合の6地域を参考に設定。

In [11]:
keys_and_settings = {
    'region'                                 : { 'input_method':'choose_from_options', 'options': [1, 2, 3, 4, 5, 6, 7, 8], 'typical': 1, 
                                                 'extra':[0, 10] },
    'main_occupant_room_floor_area'          : { 'input_method':'input_value', 'dec':0.01, 'typical': 29.81, 'min':0.01, 'max':999.99, 
                                                 'extra':[0, 100] },
    'other_occupant_room_floor_area'         : { 'input_method':'input_value', 'dec':0.01, 'typical': 51.34, 'min':0.00, 'max':999.99 },
    'total_floor_area'                       : { 'input_method':'input_value', 'dec':0.01, 'typical':120.08, 'min':0.01, 'max':999.99 },
    'input_method'                           : { 'input_method':'choose_from_options', 'options': [1, 2, 3, 4], 'typical': 1,
                                                 'exception':[2, 3, 4] },
    'insulation_type'                        : { 'input_method':'choose_from_options', 'options': [1, 2]   , 'typical': 1},
    'insulation_type_bathroom'               : { 'input_method':'choose_from_options', 'options': [1, 2, 3], 'typical': 1,
                                                 'conditions':[{'insulation_type':1}]},
    'u_value_roof'                           : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.24, 'min':0.00, 'max':10.00 },
    'u_value_wall'                           : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00 },
    'u_value_door'                           : { 'input_method':'input_value', 'dec':0.01, 'typical': 4.65, 'min':0.00, 'max':10.00 },
    'u_value_window'                         : { 'input_method':'input_value', 'dec':0.01, 'typical': 4.65, 'min':0.00, 'max':10.00 },    
    'u_value_floor_bathroom'                 : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.48, 'min':0.00, 'max':10.00,
                                                 'conditions':[{'insulation_type_bathroom':1}]},
    'u_value_floor_other'                    : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.48, 'min':0.00, 'max':10.00,
                                                 'conditions':[{'insulation_type':1}]},
    'is_psi_value_base_input'                : { 'input_method':'choose_from_options', 'options': [1, 2], 'typical': 1},
    'psi_value_earthfloor_perimeter_entrance': { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00,
                                                 'conditions':[{'is_psi_value_base_input':1}]},
    'psi_value_earthfloor_perimeter_bathroom': { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00,
                                                 'conditions':[{'insulation_type_bathroom':2, 'is_psi_value_base_input':1}]},
    'psi_value_earthfloor_perimeter_other'   : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00,
                                                 'conditions':[{'insulation_type':2, 'is_psi_value_base_input':1}]},
    'eta_d_value_window_h'                   : { 'input_method':'input_value', 'dec':0.001, 'typical': 0.630, 'min':0.000, 'max':1.000 },
    'eta_d_value_window_c'                   : { 'input_method':'input_value', 'dec':0.001, 'typical': 0.630, 'min':0.000, 'max':1.000 },
    'is_f_value_input'                       : { 'input_method':'choose_from_options', 'options': [1, 2], 'typical': 1},
    'f_value_h': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.589, 'min':0.000, 'max':1.000, 'conditions':[{'is_f_value_input':1}]},
    'f_value_c': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.864, 'min':0.000, 'max':1.000, 'conditions':[{'is_f_value_input':1}]}
}

output_cases(keys_and_settings)